In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
from collections import deque

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from neura_dual_quaternions import Quaternion

def create_3d_plot(qr = Quaternion(1,0,0,0)):
    
    plt.ioff()
    
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    fig.canvas.header_visible = False
    fig.canvas.layout.min_height = '400px'
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_axis_off()
    ax.set_box_aspect([1, 1, 1])
    ax.set_facecolor('white')
    
    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.tight_layout()
    
    start_point = [0, 0, 0]
    R_base = qr.asRotationMatrix()*1.5
    draw_frame(ax, start_point, R_base)
    
    return fig, ax

def draw_frame(ax, start_point, R):
    
    x_axis = ax.quiver(*start_point, *R[:,0], arrow_length_ratio = 0.1, linewidth = 1, color='r')
    y_axis = ax.quiver(*start_point, *R[:,1], arrow_length_ratio = 0.1, linewidth = 1, color='g')
    z_axis = ax.quiver(*start_point, *R[:,2], arrow_length_ratio = 0.1, linewidth = 1, color='b')
    return x_axis, y_axis, z_axis

def create_slider(name, start_val, min_val, max_val):
    slider_width = '98%'

    slider = widgets.FloatSlider(orientation='horizontal',description=name, value=start_val, min=min_val, max=max_val, step = 0.01, layout={'width': slider_width})
    return slider

def spherical_coordinates(azimuth, elevation):
    x = np.sin(np.pi/2 -elevation) * np.cos(azimuth)
    y = np.sin(np.pi/2 -elevation) * np.sin(azimuth)
    z = np.cos(np.pi/2 -elevation)
    vector = np.array([x, y, z])
    return vector

def create_textbox(name):
    slider_width = '70%'
    display = widgets.Text(description=name, value='', layout={'width': slider_width})
    return display

def create_quiver(ax, start_point, direction, w, c, l):
    quiver = ax.quiver(*start_point, *direction, arrow_length_ratio=0.1, linewidth = w, color = c, label = l)
    return quiver
    
def update_arc(arc, quaternion, p):
    t_values = np.linspace(0, 1, 50)
    arc_points = np.array([(Quaternion.slerp(Quaternion(1,0,0,0), quaternion, t)*p*Quaternion.slerp(Quaternion(1,0,0,0), quaternion, t).inverse()).getVector().flatten() for t in t_values])

    arc.set_data(arc_points[:,0], arc_points[:,1])
    arc.set_3d_properties(arc_points[:,2])
    

def rpy_from_R(R):  
    # calculation of roll pitch yaw angles from rotation matrix
    yaw = np.arctan2(R[1, 0], R[0, 0])
    pitch = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
    roll = np.arctan2(R[2, 1], R[2, 2])
    
    return roll, pitch, yaw

def Rot_rpy(roll, pitch, yaw):
    
    # Roll matrix
    Rx = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])
    
    # Pitch matrix
    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])
    
    # Yaw matrix
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])
    
    # Combined rotation matrix
    R = Rz@Ry@Rx
    return R


<h1> Introduction to Quaternions</h1>

This section aims to give insight into some mathematical foundations used throughout the Thesis. More specifically, it presents the main concepts and operations of quaternions, which are later extended to unit dual quaternions to use the benefits of quaternions in the realm of Transformations.

Hamilton introduced quaternions in the nineteenth century which can be viewed as an extension of the complex number theory. Instead of only one imaginary unit as for a complex number, quaternions are defined with three imaginary components $i$, $j$, and $k$, which possess the following properties <cite id="lunr8"><a href="#zotero%7C16222978%2FH9XC4M77">(Hamilton, 1844)</a></cite>:

$$
-ji = ij = k, \qquad -kj = jk = i, \qquad -ik = ki = j, \\
i^2 = j^2 = k^2 = -1
$$

When discovered by Hamilton, quaternions were just a theoretical concept, but in recent years they have been broadly used to represent both orientation and rotation in three-dimensional space due to their advantages over classical Euler angle representations <cite id="9zmqf"><a href="#zotero%7C16222978%2F7MLS2H2W">(Perumal, 2011)</a></cite>, <cite id="rbr1d"><a href="#zotero%7C16222978%2FZV86QVV9">(Alaimo et al., 2013)</a></cite>. Unlike traditional vector rotations that suffer from issues like gimbal lock, quaternions offer a robust and efficient way to represent spatial orientations and rotations. Their ability to interpolate between orientations <cite id="ndpzf"><a href="#zotero%7C16222978%2FM23ULJLX">(Shoemake, 1985)</a></cite> and the ease with which they handle compound rotations make them indispensable in modern robotics.

A single quaternion is expressed as: 

$$
q = w + xi + yj + zk
$$

where $w$, $x$, $y$ and $z$ are real numbers and $i$, $j$ and $k$ are the quaternion units. Similar to complex numbers, a quaternion can be divided into <i>real</i> and <i>complex</i> parts. Here, $x$, $y$, $z$ are the complex parts, whereas $w$ is called real part. Different Notations are common and used throughout the Thesis. The most common is the vector notation, where the complex part of the quaternion is condensed into the so-called <i>complex vector</i> $\vec{v} = (x, y, z)$. The following list will give a short overview:

<h3>Different Quaternion Notations:</h3>
<ul>
    <li>Quaternion: $ q = w + xi + yj + zk $</li>
    <li>Quaternion: $ q = (w, x, y, z) $</li>
    <li>Quaternion: $ q = (w, \vec{v}) $</li>
</ul>

In the scope of this thesis a quaternion package was developed <cite id="qm06h"><a href="#zotero%7C16222978%2FAGXR4PGH">(Temminghoff, 2023)</a></cite> to be able to use quaternions in the further context of the Thesis. To introduce this package a small example of the basic constructor of a quaternion is given:

In [2]:
#Example: identity quaternion
quat = Quaternion(1,0,0,0)

print(quat)

Quaternion(1.000, 0.000, 0.000, 0.000)


<h2>Orientation and Rotation representation with Unit Quaternions </h2>

A quaternion must be unit length to represent orientation and rotation, which is a benefit over the orientation representation with rotation matrices, meaning $\|q\| = 1$. The euclidean norm of a unit quaternion is defined as: 

$$
\|q\| = \sqrt{w^2 + x^2 + y^2 + z^2}.
$$

Quaternions with unit length are called <strong>Unit Quaternions</strong> and can be thought of as points on the unit four-dimensional hypersphere surface, which spans the curved space of $\mathcal{S}^3$. This characteristic allows unit quaternions to represent orientations and rotations as they lose their ability to scale. This is analog to the fact that the determinant of a rotation matrix is one. Another analogy is that the concatenation of Quaternions is not commutative, which means that the order of multiplication matters to the final orientation

$$
    {}_0q^1 =q_x \otimes q_y \neq q_y \otimes q_x
$$

Note: $\otimes$ denotes the quaternion multiplication 

To use Quaternions for Orientation representation, most commonly the axis angle representation of rotation / orientation is used. Here a unit quaternion $q \in \mathcal{S}^3$ is associated with $\theta \tilde{r} \in \mathbb{R}^3$ and can be derived via the exponential map as,

$$
q := e^{\frac{\theta}{2}\tilde{r}} = \left(\cos\frac{\theta}{2},  \quad \sin\frac{\theta}{2} \tilde{r} \right).
$$

For this intuitive way to describe rotation, the angle $\theta$ describes the rotation angle around the unit-length rotation axis $\tilde{r}$. The exponential map is a <i>Lie Theoretic</i> concept and can be further studied with great detail in the excellent work <cite id="1rst5"><a href="#zotero%7C16222978%2FGF7XZFAM">(Shahidi, 2023)</a></cite>. Delving too deep into the mathematical foundations of lie theory and Clifford algebra would exceed the scope of this Thesis.

To explain the exponential mapping in more detail, an interactive code example is given, here a rotation axis can be manipulated and a coordinate system, subsequently called frame can be rotated around the rotation axis $\tilde{r}$ via the setting of an angle $\theta$.

In [33]:
fig, ax = create_3d_plot()
quaternion_display = create_textbox("Quaternion")
angle_slider = create_slider("theta", 0, -2*np.pi, 2*np.pi)
azimuth_slider = create_slider("azimuth", 0, -2*np.pi, 2*np.pi)
elevation_slider = create_slider("elevation", 0, -np.pi, np.pi)
#angle_slider, azimuth_slider, elevation_slider = create_quaternion_sliders()

rotation_axis = create_quiver(ax, [0,0,0], [1,0,0],1, 'grey', 'rotation axis')
imaginary_part = create_quiver(ax, [0,0,0], [0,0,0], 3, 'k', 'complex vector')
x_axis, y_axis, z_axis = draw_frame(ax, [0,0,0], np.eye(3))
point_w = ax.scatter(0,0,1,s = 30, c = 'k', label = "real part")
ax.legend()
# Update function for the sliders
def update_plot(change):
    global rotation_axis, imaginary_part, x_axis, y_axis, z_axis, point_w

    rotation_axis.remove()
    imaginary_part.remove()
    x_axis.remove()
    y_axis.remove()
    z_axis.remove()
    point_w.remove()
    
    angle = angle_slider.value
    direction = spherical_coordinates(azimuth_slider.value, elevation_slider.value)
    
    # construct unit quaternion from axis angle
    quaternion = Quaternion.exp(Quaternion(0, *0.5*angle*direction))    
    
    # update displays
    quaternion_display.value = str(quaternion)
    
    # update the drawn vectors
    rotation_axis = create_quiver(ax, [0,0,0], direction, 1, 'grey', 'rotation axis')
    imaginary_part = create_quiver(ax, [0,0,0], [quaternion.x, quaternion.y, quaternion.z], 3, 'k', 'complex vector')
    x_axis, y_axis, z_axis = draw_frame(ax, [0,0,0], quaternion.asRotationMatrix())
    
    point_w = ax.scatter(0,0,quaternion.w, s = 30, c = "k", label = "real part")
    fig.canvas.draw()
    fig.canvas.flush_events()

    
angle_slider.observe(update_plot, names = 'value')
azimuth_slider.observe(update_plot, names='value')
elevation_slider.observe(update_plot, names='value')

widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.VBox([quaternion_display, angle_slider, azimuth_slider, elevation_slider]),
    pane_heights=[0, 3, 1]
)

AppLayout(children=(VBox(children=(Text(value='', description='Quaternion', layout=Layout(width='70%')), Float…

As seen in the plot, the complex vector of the quaternion is the scaled rotation axis which is dependant on the angle of rotation and the real part moves according to the angle s.t. the unit length property is satisfied. This  gives an inuitive way to think about quaternions, as they are often deemed to be too complex to be visualized properly and unintuitive in application. Besides an interactive visualization and intuitive understanding of the lie-theoretic exponential map another important characteristic of quaternions can be seen: The <strong>antipodal property</strong>. 

This property is described by the fact the a quaternion $q = (w, x, y, z)$ and its antipodal counterpart $-q = (-w, -x, -y, -z)$ show the same orientation, i.e. $R(q) = R(-q)$. Note that conversion from a quaternion to rotation matrix is omitted in this thesis, a detailed and robust implementation and can be found in the package:  <cite id="qm06h"><a href="#zotero%7C16222978%2FAGXR4PGH">(Temminghoff, 2023)</a></cite>. 

Even though the final orientation of the given quaternion $q$ matches with $-q$, the rotation to reach this orientation differs. To show what is meant, the given examples interpolated from the identiy orientation to a given target quaternion $q_{target}$ and its given antipodal quaternion $-q_{target}$.

In [40]:
#Example: Antipodal Property
q_start = Quaternion(1,0,0,0)
q_target = Quaternion(.900, 0.150, 0.000, 0.15).normalize()
q_target_antipodal = -1.0*q_target

fig, ax = create_3d_plot(q_target)

s_slider = create_slider("s", 0, 0, 1)
x_axis1, y_axis1, z_axis1 = draw_frame(ax, [0,0,0], q_start.asRotationMatrix())
x_axis2, y_axis2, z_axis2 = draw_frame(ax, [0,0,0], q_start.asRotationMatrix())

def update_plot(change):
    global x_axis1, y_axis1, z_axis1
    global x_axis2, y_axis2, z_axis2
    
    x_axis1.remove()
    y_axis1.remove()
    z_axis1.remove()

    x_axis2.remove()
    y_axis2.remove()
    z_axis2.remove()
    
    q_interpolated1 = Quaternion.slerp(q_start, q_target, s_slider.value)    
    q_interpolated2 = Quaternion.slerp(q_start, q_target_antipodal, s_slider.value)  
    
    x_axis1, y_axis1, z_axis1 = draw_frame(ax, [0,0,0], q_interpolated1.asRotationMatrix())
    x_axis2, y_axis2, z_axis2 = draw_frame(ax, [0,0,0], q_interpolated2.asRotationMatrix())

    fig.canvas.draw()
    fig.canvas.flush_events()

    
s_slider.observe(update_plot, names = 'value')

widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.VBox([s_slider]),
    pane_heights=[0, 3, 1]
)

AppLayout(children=(VBox(children=(FloatSlider(value=0.0, description='s', layout=Layout(width='98%'), max=1.0…

From the interactive plot it is clearly visible that both quaternions start and end in the same orientation, but take different paths to reach this point. The rotation $q_{start} \to q_{target}$ takes she shortest path around the aperent, constant rotation axis, where as $q_{start} \to -q_{target}$ takes the long path to reach the target orientation. It can be explained by the fact that a negative rotation around the inverted rotation axis yields the same orientation as a positive orientation around the normal rotation axis, i.e $\theta \tilde{r} = (-\theta)(-\tilde{r})$, which means that the quaternions $q_{target}$ and $-q_{target}$ lie on different hemispheres on the hypersphere surface $\mathcal{S}^3.$ This property is unique to unit quaternions and can be leveraged in robotic motion-planning, for more complex and dexterous motions.

<h2> Pure Quaternions </h2>

Besides representing orientation and rotation in three dimensional space, Unit Quaternions can be used to rotate vectors in $\mathbb{R}^3$. For this we need to introduce another form of Quaternion, called <strong>Pure Quaternion</strong>.

<h1> Literature</h1>
